In [ ]:
!nvidia-smi # Check whether your system has a GPU

In [1]:
!pip list

Package                     Version              Editable project location
--------------------------- -------------------- -----------------------------------------------------------------------------------
about-time                  3.1.1
absl-py                     1.2.0
addict                      2.4.0
aiohttp                     3.8.1
aiosignal                   1.3.1
albumentations              1.2.1
alive-progress              2.4.1
anyio                       3.6.2
appdirs                     1.4.4
argon2-cffi                 21.3.0
argon2-cffi-bindings        21.2.0
async-timeout               4.0.2
asyncer                     0.0.1
asynctest                   0.13.0
attrs                       21.4.0
backcall                    0.2.0
backgroundremover           0.1.9
bbox-visualizer             0.1.0
beautifulsoup4              4.11.1
bleach                      5.0.1
brotlipy                    0.7.0
cachetools                  5.2.0
caerus                      0.1.8
certi

### Dataset Download

In [ ]:
dataset_url = "https://drive.google.com/file/d/1Ru1dLP8vXUnVbDiwQqf--qyiYkHd9C_1/view?usp=share_link"
!gdown 1Ru1dLP8vXUnVbDiwQqf--qyiYkHd9C_1

In [ ]:
import shutil
shutil.unpack_archive('flower_photos.tgz','./')

In [ ]:
!rm -rf  flower_photos/LICENSE.txt

In [ ]:
import os
names = os.listdir('flower_photos')
os.makedirs(os.path.join('flower_photos/train/'),exist_ok=True)
for name in names:
    shutil.move(f'flower_photos/{name}',f'flower_photos/train/')
    os.makedirs(os.path.join('flower_photos/validation/',name),exist_ok=True)

In [ ]:
import shutil
train_names = os.listdir('flower_photos/train/')
for train_name in train_names:
    file_names = sorted(os.listdir(os.path.join('flower_photos/train/',train_name)))
    for file_name in file_names[-100:]:
        shutil.move(os.path.join('flower_photos/train/',train_name,file_name),os.path.join('flower_photos/validation/',train_name))

# Import packages

In [ ]:
import numpy as np
import cv2
import torch 
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from tqdm import tqdm

numpy 
- A scientific computing library
- Supports array operations and manipulations
- Built-in functions like array sum, diff, mean, average, min, max etc.

cv2
- To read and write images
- To perform basic vision algorithms like RGB to HSV, Gaussian Blur, Dilation, Histogram, Shape detection etc.

torch
- The open source ML framework for deep learning
- Has a pythonic interface

torchvision
- consists of popular datasets, model architectures, and common image transformations for computer vision
- Contains built in data augmentations

matplotlib
- It is a data visualization library for python
- It supports graphical plotting and viewing images and other data

nn
- The base module with which we can create and train neural nets
- Provides built-in classes for common neural network layers like Conv, Pooling, Activations, Normalizations, Dropout etc.

optim
- It is a package for implementing various optimization algorithms for model training
- Built-in optimizers like SGD, Adam, Adagrad, RMSprop etc.

tqdm
- It is a library used to display a progress bar for loops

# Preprocess inputs

In [ ]:
transform = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor()]) # Define the set of transformations to be applied on the input data

In [ ]:
# transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,)),])

ToTensor()
 - converts the image with a pixel range of [0, 255] to a PyTorch FloatTensor of shape (C, H, W) with a range [0.0, 1.0]

Normalize() 
- output[channel] = (input[channel] - mean[channel]) / std[channel]

- Normalization helps get data within a range and reduces the skewness which helps learn faster and better. 
- Normalization can also tackle the diminishing and exploding gradients problems.

# Custom Dataloaders using Image Folder function

- Combines a dataset and a sampler
- Provides an iterable over the given dataset
- Supports single- or multi-process loading
- Customizing loading order
- Automatic batching

In [ ]:
trainset = datasets.ImageFolder(root ='flower_photos/train',transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True)

In [ ]:
valset = datasets.ImageFolder(root ='flower_photos/validation',transform=transform)
valloader = torch.utils.data.DataLoader(valset, batch_size=8, shuffle=True)

In [ ]:
dataiter = iter(trainloader)

Now that we have an iterator, let's visualize our data

In [ ]:
images, gt_labels = dataiter.next() # The first minibatch of data

In [ ]:
print(images.shape)
print(gt_labels.shape)

In [ ]:
 # Visualize an image sample
plt.imshow((images[2].squeeze()).permute(1,2,0).numpy())

In [ ]:
classes = trainset.classes

In [ ]:
# Visualize all images in the minibatch
figure = plt.figure()
total_samples = 8
for index in range(1,total_samples+1):
    ax = plt.subplot(2, 4, index)
    ax.set_xlabel(classes[gt_labels[index-1].item()])
    # plt.axis('off')
    plt.imshow(images[index-1].squeeze().permute(1,2,0).numpy())

In [ ]:
# Visualize the Ground truth labels
print(gt_labels)

In [ ]:
# Visualize the onehot encoding of the labels
onehot_labels = nn.functional.one_hot(gt_labels, num_classes=10)
print(onehot_labels)

# Let's build our AlexNet
![](assets/AlexNet.jpeg)

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 10, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.flatten = nn.Flatten()
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
model = AlexNet()

In [ ]:
print(model)

### Model Training

In [ ]:
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)
epochs = 100
pbar = tqdm(range(epochs))
best_accuracy = 0.0

In [ ]:
time_init = time() # record the time at which training started

for e in pbar:
    running_loss = 0
    for images, gt_labels in tqdm(trainloader):
        images = images.cuda()
        gt_labels = gt_labels.cuda()
    
        # Training pass
        optimizer.zero_grad() # reset the gradients of model weights
        
        output = model(images)
        loss = criterion(output, gt_labels)
        
        # Calculate the gradients of the learnable parameters
        loss.backward()
        
        # Modify the model weights as per the gradients
        optimizer.step()
        
        running_loss += loss.item()
    average_loss = running_loss/len(trainloader)
    
    # perform validation
    correct_count = 0
    total_count = 0
    for images,gt_labels in valloader:
      pred_probs = model(images.cuda())
      predicted_labels = torch.argmax(pred_probs, dim=-1)

      for i in range(len(gt_labels)):
        if predicted_labels[i]==gt_labels[i]:
          correct_count = correct_count+1
        total_count = total_count+1       

    accuracy = correct_count/total_count
    
    # save the model weights
    if accuracy>=best_accuracy:
      best_accuracy = accuracy
      torch.save(model, './best_flower_model.pt')
    torch.save(model, './last_flower_model.pt') 

    print(f"\nEpoch {e} - Training loss: {average_loss}, val accuracy : {accuracy}")
print(f"Training Time (in minutes) = {(time()-time_init)/60}")

### Model Validation

In [ ]:
images, gt_labels = next(iter(valloader))

with torch.no_grad():
    pred_probs = model(images.cuda())

predicted_labels = torch.argmax(pred_probs, dim=-1)

### Result Visualisation

In [ ]:
# Visualize all images in the minibatch
figure = plt.figure()
total_samples = 8
for index in range(1,total_samples+1):
    ax = plt.subplot(2, 4, index)
    ax.set_xlabel(classes[gt_labels[index-1].item()])
    # plt.axis('off')
    plt.imshow(images[index-1].squeeze().permute(1,2,0).numpy())

In [ ]:
print(f"Ground truth labels : {gt_labels}")

In [ ]:
print(f"Predicted labels : {predicted_labels}",)

In [ ]:
# Visualize all images in the minibatch
figure = plt.figure()
total_samples = 8
for index in range(1,total_samples+1):
    ax = plt.subplot(2, 4, index)
    ax.set_xlabel(classes[predicted_labels[index-1].item()])
    # plt.axis('off')
    plt.imshow(images[index-1].squeeze().permute(1,2,0).numpy())

In [ ]:
correct_count = 0
total_count = 0
for i in range(len(gt_labels)):
  img = images[i]
  if predicted_labels[i]==gt_labels[i]:
    correct_count = correct_count+1
  total_count = total_count+1
accuracy = correct_count/total_count
print(f"The accuracy on the minibatch is : {accuracy}")

In [ ]:
correct_count = 0
total_count = 0

for images,gt_labels in valloader:
  pred_probs = model(images.cuda())
  predicted_labels = torch.argmax(pred_probs, dim=-1)

  for i in range(len(gt_labels)):
    if predicted_labels[i]==gt_labels[i]:
      correct_count = correct_count+1
    total_count = total_count+1       

accuracy = correct_count/total_count
print(f"Number of validation images = {total_count}\n")
print(f"Model Accuracy = {accuracy}")